In [ ]:
import praw
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [ ]:
# Carregar o modelo treinado
model = load_model(r"..\modelos\modelo_rnn.h5")  # Certifique-se de usar o caminho correto para o seu modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Carregar o tokenizer
with open("tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)


In [ ]:
# Função para classificar o sentimento de um texto
def classify_sentiment(text, tokenizer, model):
    # Tokenização e vetorização
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=100)
    # Prever o sentimento usando o modelo treinado
    prediction = model.predict(padded_sequence)[0][0]
    if prediction >= 0.5:
        return "Positivo"
    else:
        return "Negativo"


In [ ]:
# Configurar as credenciais para acessar a API do Reddit
reddit = praw.Reddit(
    client_id="0LUIMHwzq6iTBcF4F4zGpQ",
    client_secret="AN90R4CXtXjCpEfXEEVCIKIjReY0NA",
    user_agent="aps",
)

In [ ]:
# Lista de tópicos de interesse
topics = ["deforestation", "forestfires", "floods", "rain", "riverpollution"]


In [ ]:
# Criar uma lista para armazenar os dados
data = []

In [ ]:
# Iterar sobre os posts do Reddit
for topic in topics:
    for submission in reddit.subreddit("all").search(topic, sort="hot", time_filter="week", limit=20):
        if submission.selftext.strip() != "" or submission.url.strip() != "":
            # Concatenar o título e o conteúdo do post
            text = submission.title + " " + submission.selftext
            # Classificar o sentimento do texto
            sentiment = classify_sentiment(text, tokenizer, model)
            # Adicionar os dados à lista
            data.append([submission.id, submission.title, submission.selftext, sentiment, submission.subreddit, 
                         submission.author.name, submission.url, submission.score, submission.num_comments, 
                         submission.created_utc])

In [ ]:
# Criar um DataFrame com os dados
df = pd.DataFrame(data, columns=['id', 'title', 'content', 'sentiment', 'subreddit', 'author', 'url', 'score', 
                                 'num_comments', 'created_utc'])

In [ ]:
# Salvar o DataFrame em um arquivo CSV
csv_file_path = r"..\data\reddit_posts.csv"
df.to_csv(csv_file_path, index=False)
print("Dados salvos com sucesso no arquivo CSV.")